In [5]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.cm as cm
import numpy as np
import matplotlib.pyplot as plt
import csv
import numpy as np
import os


def cart2sph(xyz,center):
    x = np.subtract(xyz[:,0],center[0])
    y = np.subtract(xyz[:,1],center[1])
    z = np.subtract(xyz[:,2],center[2])
    rtp = np.zeros(xyz.shape)
    r = np.sqrt(np.power(x,2)+np.power(y,2)+np.power(z,2))
    theta = np.arccos(np.divide(z,r))
    #phi = np.arctan(np.divide(y,x))
    phi = np.arctan2(y,x)
    rtp[:,0] = r
    rtp[:,1] = theta
    rtp[:,2] = phi
    return rtp
    
tx_loc = [578.121009649043120, 557.409398267150100, 10.000000000000000]

pg_file_path = r'D:\Remcom Simulations\Mimo_Tutorial\Omni BS Omni Rx Route\MIMO_Example_Rosslyn.pg.t001_06.r005.p2m'
pg_raw_data = open(pg_file_path, 'r')
pg_reader = csv.reader(pg_raw_data, delimiter=' ', quoting=csv.QUOTE_NONE)
all_pg = list(pg_reader)
all_location_data = np.array(all_pg[2:]).astype('float')
nrx = all_location_data.shape[0]
print('total number of rx: ', nrx)
study_area_dir = r'D:\Remcom Simulations\Mimo_Tutorial\MIMO 16x16 Omni Rx Route'
valid_rx_pt = [];
for i in range(nrx-1):
    datarow = np.zeros(6);
    h_path = 'hmatrix\\hmatrix.txSet004.txPt001.rxSet005.rxPt{0}.inst001.csv'.format(str(i+1).zfill(3))
    abs_h_path = os.path.join(study_area_dir, h_path)
    raw_h_data = open(abs_h_path, 'rt')
    h_reader = csv.reader(raw_h_data, delimiter=',', quoting=csv.QUOTE_NONE)
    h = list(h_reader)
    h = np.array(h).astype('float')
    if np.count_nonzero(h) != 0:
        valid_rx_pt.append(i)
nvalid_rx_pt = len(valid_rx_pt) 
print(nvalid_rx_pt)
data = np.zeros((nvalid_rx_pt,7))
channels = np.zeros((nvalid_rx_pt,2,256))
for j in range(nvalid_rx_pt):
    i = valid_rx_pt[j]
    #MISO original
    #rel_path = 'H\\hmatrix\\hmatrix.txSet011.txPt001.rxSet046.rxPt{0}.inst001.csv'.format(str(i+1).zfill(3))
    #MISO rotated
    h_path = 'hmatrix\\hmatrix.txSet004.txPt001.rxSet005.rxPt{0}.inst001.csv'.format(str(i+1).zfill(3))
    abs_h_path = os.path.join(study_area_dir, h_path)
    raw_h_data = open(abs_h_path, 'rt')
    h_reader = csv.reader(raw_h_data, delimiter=',', quoting=csv.QUOTE_NONE)
    h = list(h_reader)
    h = np.array(h).astype('float')
#     h = h[:,0]+1j*h[:,1]
#     h=h.reshape((256,1))
    channels[j,0,:] = h[:,0]
    channels[j,1,:] = h[:,1]
    

total number of rx:  3145
3144


In [20]:
from scipy.io import savemat
processed_data = {'channel':channels, 'tx_loc':tx_loc, 'valid_rx_idx':valid_rx_pt,'rx_loc_all':all_location_data[:,1:4]}
savemat('route_data.mat',processed_data)

In [21]:
processed_data['rx_loc_all']


array([[596.182 , 661.779 ,  40.2026],
       [596.179 , 661.679 ,  40.2092],
       [596.176 , 661.579 ,  40.2158],
       ...,
       [607.149 , 463.182 ,  48.9735],
       [607.05  , 463.164 ,  48.9879],
       [606.952 , 463.146 ,  49.0023]])

In [94]:
#import modules
import re
import xml.etree.ElementTree as ET
import os
import numpy as np

study_area_placement = re.compile(r"(end_<studyarea>)\n(begin_<feature>)")
study_area_section = re.compile(r"begin_<studyarea>.*?end_<studyarea>", re.DOTALL)

study_area = "D:\Remcom Simulations\Mimo_Tutorial\MIMO_Example_Rosslyn.Test.xml"

#txrx_move = True
#txrx_index = "1"

#study_area_directory = os.path.(study_area)
study_area_filename = os.path.basename(study_area)
# split_study_area = os.path.splitexdirnamet(study_area)
# study_area_filename_split = study_area_filename.split(".")

    
# with open(study_area) as f:
#     study_area_content = f.readlines()

#     #modify .xml to change the study area and object used    
#     tree = ET.parse(study_area)
#     #change the output location by adding an index to the directory
#     for element_outputlocation in tree.find("Job/OutputLocation"):
#         element_outputlocation.set("Value", element_outputlocation.get("Value") + " " + str(index))
#         print(element_outputlocation.attrib)
#     #add index to the output prefix to identify files
#     for element_outputprefix in tree.find("Job/OutputPrefix"):
#         element_outputprefix.set("Value", element_outputprefix.get("Value") + " " + str(index))
#         print(element_outputprefix.attrib)
#     #add index to the sqlite database to identify it
#     for element_databaselocation in tree.find("Job/PathResultsDatabase/PathResultsDatabase/Filename/FileDescription/Filename"):
#         element_databaselocation.set("Value", "./" + element_outputlocation.get("Value")+ "/" + element_outputprefix.get("Value") + "." + element_outputlocation.get("Value") + ".sqlite")
#         print(element_databaselocation.attrib)
#     new_study_area_path = split_study_area[0] + " " + str(index) + split_study_area[1]
#     tree.write(open(new_study_area_path, "w"), encoding='unicode')
        
#     #modify .setup to add the new study area to the project
#     #duplicate existing study area and increment index
#     setup_content = re.sub(r"FirstAvailableStudyAreaNumber.+", "FirstAvailableStudyAreaNumber " + str(index), setup_content)
#     for match in study_area_section.finditer(setup_content):
#         study_area_match = "\n"+match.group(0)+"\n"
#         #TODO this regex doesn't match well
#         study_area_match = re.sub(r"(begin_<studyarea>.+)", r"\g<0> " + str(index), study_area_match)
#         study_area_match = re.sub(r"(StudyAreaNumber\s)(\d+)", r"StudyAreaNumber " + str(index), study_area_match)
#     #insert new study area after the last one
#     for study_area_placement_bounds in study_area_placement.finditer(setup_content):
#         study_area_match_top, study_area_match_bottom = study_area_placement_bounds.groups()
#     setup_content = study_area_placement.sub(study_area_match_top + study_area_match + study_area_match_bottom, setup_content)
#     out_object = open(setup,'w')
#     out_object.writelines(setup_content)
#     out_object.close()

#     i += 1
#     index += 1


In [95]:
tree = ET.parse(study_area)

In [73]:
for element in tree.find("Job/Scene/Scene/AntennaList/AntennaList/Antenna/MIMO/MIMOElement/MIMOElement/Offset/Tuple3/X"):
    print(element.attrib['Value'])

0


In [75]:
for element in tree.iter('Job'):
    print(element.tag, element.attrib)

Job {}


In [102]:
ele_pos = np.zeros((3,256))
i = 0
for element in tree.iterfind('Job/Scene/Scene/AntennaList/AntennaList/Antenna/MIMO/MIMOElement/MIMOElement/Offset/Tuple3/X/'):
#     print(i,element.attrib['Value'])
    ele_pos[0,i] = element.attrib['Value']
    i+=1
i = 0
for element in tree.iterfind('Job/Scene/Scene/AntennaList/AntennaList/Antenna/MIMO/MIMOElement/MIMOElement/Offset/Tuple3/Y/'):
#     print(i,element.attrib['Value'])
    ele_pos[1,i] = element.attrib['Value']
    i+=1
i = 0
for element in tree.iterfind('Job/Scene/Scene/AntennaList/AntennaList/Antenna/MIMO/MIMOElement/MIMOElement/Offset/Tuple3/Z/'):
#     print(i,element.attrib['Value'])
    ele_pos[2,i] = element.attrib['Value']
    i+=1
    

In [89]:
for loc in element.getchildren():
    print(loc)

<Element 'X' at 0x000002A4E15BE4F8>
<Element 'Y' at 0x000002A4E15BE598>
<Element 'Z' at 0x000002A4E15BE638>


In [93]:
loc.

<function Element.getchildren()>

In [106]:
ele_pos[2,:]

array([-0.0401508 , -0.0347973 , -0.0294439 , -0.0240905 , -0.018737  ,
       -0.0133836 , -0.00803016, -0.00267672,  0.00267672,  0.00803016,
        0.0133836 ,  0.018737  ,  0.0240905 ,  0.0294439 ,  0.0347973 ,
        0.0401508 , -0.0401508 , -0.0347973 , -0.0294439 , -0.0240905 ,
       -0.018737  , -0.0133836 , -0.00803016, -0.00267672,  0.00267672,
        0.00803016,  0.0133836 ,  0.018737  ,  0.0240905 ,  0.0294439 ,
        0.0347973 ,  0.0401508 , -0.0401508 , -0.0347973 , -0.0294439 ,
       -0.0240905 , -0.018737  , -0.0133836 , -0.00803016, -0.00267672,
        0.00267672,  0.00803016,  0.0133836 ,  0.018737  ,  0.0240905 ,
        0.0294439 ,  0.0347973 ,  0.0401508 , -0.0401508 , -0.0347973 ,
       -0.0294439 , -0.0240905 , -0.018737  , -0.0133836 , -0.00803016,
       -0.00267672,  0.00267672,  0.00803016,  0.0133836 ,  0.018737  ,
        0.0240905 ,  0.0294439 ,  0.0347973 ,  0.0401508 , -0.0401508 ,
       -0.0347973 , -0.0294439 , -0.0240905 , -0.018737  , -0.01

In [107]:
from scipy.io import savemat
processed_data = {'channel':channels, 'tx_loc':tx_loc, 'valid_rx_idx':valid_rx_pt,'rx_loc_all':all_location_data[:,1:4],'tx_ele_pos':ele_pos}
savemat('route_data.mat',processed_data)

In [109]:
processed_data['tx_ele_pos'].shape

(3, 256)